In [309]:
import pandas as pd
import numpy as np

In [334]:
df = pd.read_csv('click_stream.csv', encoding = "UTF-8", header=None, names = ['user_id', 'page', 'event_date'])

In [335]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142082 entries, 0 to 142081
Data columns (total 3 columns):
user_id       142082 non-null int64
page          142082 non-null object
event_date    142082 non-null object
dtypes: int64(1), object(2)
memory usage: 3.3+ MB


In [336]:
df.head()

,user_id,page,event_date
0,313593,1_home_page,2015-02-26
1,468315,1_home_page,2015-02-21
2,264005,1_home_page,2015-03-25
3,290784,1_home_page,2015-03-14
4,639104,1_home_page,2015-01-03


In [328]:
df.groupby(['page', 'event_date'])['user_id'].count()

page                         event_date
1_home_page                  2015-01-01    712
                             2015-01-02    721
                             2015-01-03    760
                             2015-01-04    713
                             2015-01-05    754
                                          ... 
4_payment_confirmation_page  2015-04-25      1
                             2015-04-26      1
                             2015-04-27      1
                             2015-04-28      1
                             2015-04-30      1
Name: user_id, Length: 469, dtype: int64

In [338]:
df.loc[:,'month'] = df['event_date'].apply(lambda x: x[:7])
df.head(5)

,user_id,page,event_date,month
0,313593,1_home_page,2015-02-26,2015-02
1,468315,1_home_page,2015-02-21,2015-02
2,264005,1_home_page,2015-03-25,2015-03
3,290784,1_home_page,2015-03-14,2015-03
4,639104,1_home_page,2015-01-03,2015-01


In [391]:
pvt = pd.pivot_table(df, values=['user_id'], index=['page'], columns=['month'], aggfunc=len)
pvt

user_id                        
month                       2015-01 2015-02 2015-03 2015-04
page                                                       
1_home_page                   22600   22600   22600   22600
2_search_page                 13554   13687    8879    9080
3_payment_page                 2390    2412     631     597
4_payment_confirmation_page     189     173      44      46

In [397]:
(pvt / pvt.iloc[0] * 100).round(2)

user_id                        
month                       2015-01 2015-02 2015-03 2015-04
page                                                       
1_home_page                  100.00  100.00  100.00  100.00
2_search_page                 59.97   60.56   39.29   40.18
3_payment_page                10.58   10.67    2.79    2.64
4_payment_confirmation_page    0.84    0.77    0.19    0.20

In [396]:
(pvt[1:] / pvt[1:].iloc[0] * 100).round(2)

user_id                        
month                       2015-01 2015-02 2015-03 2015-04
page                                                       
2_search_page                100.00  100.00  100.00  100.00
3_payment_page                17.63   17.62    7.11    6.57
4_payment_confirmation_page    1.39    1.26    0.50    0.51

In [339]:
df2 = df.groupby(['month','page'])['user_id'].count().reset_index().rename(columns={'user_id':'page_click_count'})

In [377]:
df2.loc[df2['page'] == '1_home_page', 'tmp'] = df2['page_click_count']
df2['level_1'] = df2['tmp'].ffill()
# df2['rate'] = (df2['page_click_count'] / df2['level_1']) * 100
df2 = df2.assign(ratio=lambda x: x['page_click_count'] / x['tmp'],
                 percent=lambda x: x['ratio'] * 100)
del df2['tmp']
df2['percent'] = df2['percent'].round(decimals=2)
df2

,month,page,page_click_count,ratio,percent
0,2015-01,1_home_page,22600,1.000000,100.00
1,2015-01,2_search_page,13554,0.599735,59.97
2,2015-01,3_payment_page,2390,0.105752,10.58
3,2015-01,4_payment_confirmation_page,189,0.008363,0.84
4,2015-02,1_home_page,22600,1.000000,100.00
5,2015-02,2_search_page,13687,0.605619,60.56
6,2015-02,3_payment_page,2412,0.106726,10.67
7,2015-02,4_payment_confirmation_page,173,0.007655,0.77
8,2015-03,1_home_page,22600,1.000000,100.00
9,2015-03,2_search_page,8879,0.392876,39.29


In [378]:
pd.pivot_table(df2, values=['page_click_count', 'percent'], index = 'page', columns=['month'], )

page_click_count                         percent  \
month                                2015-01 2015-02 2015-03 2015-04 2015-01   
page                                                                           
1_home_page                            22600   22600   22600   22600  100.00   
2_search_page                          13554   13687    8879    9080   59.97   
3_payment_page                          2390    2412     631     597   10.58   
4_payment_confirmation_page              189     173      44      46    0.84   

                                                     
month                       2015-02 2015-03 2015-04  
page                                                 
1_home_page                  100.00  100.00  100.00  
2_search_page                 60.56   39.29   40.18  
3_payment_page                10.67    2.79    2.64  
4_payment_confirmation_page    0.77    0.19    0.20

In [379]:
df3 = df2[df2['page'] != '1_home_page']
df3.loc[df3['page'] == '2_search_page', 'tmp'] = df3.loc[:,'page_click_count']
df3.loc[:,'tmp'] =  df3['tmp'].ffill()
df3 = df3.assign(ratio=lambda x: x['page_click_count'] / x['tmp'],
                 percent =lambda x: x['ratio'] * 100)
del df3['tmp']
df3['percent'] = df3['percent'].round(decimals=2)
df3

,month,page,page_click_count,ratio,percent
1,2015-01,2_search_page,13554,1.000000,100.00
2,2015-01,3_payment_page,2390,0.176332,17.63
3,2015-01,4_payment_confirmation_page,189,0.013944,1.39
5,2015-02,2_search_page,13687,1.000000,100.00
6,2015-02,3_payment_page,2412,0.176226,17.62
7,2015-02,4_payment_confirmation_page,173,0.012640,1.26
9,2015-03,2_search_page,8879,1.000000,100.00
10,2015-03,3_payment_page,631,0.071067,7.11
11,2015-03,4_payment_confirmation_page,44,0.004956,0.50
13,2015-04,2_search_page,9080,1.000000,100.00


In [381]:
df3.pivot(index='page', columns='month', values=['page_click_count', 'percent'])

page_click_count                          percent  \
month                                2015-01  2015-02 2015-03 2015-04 2015-01   
page                                                                            
2_search_page                        13554.0  13687.0  8879.0  9080.0  100.00   
3_payment_page                        2390.0   2412.0   631.0   597.0   17.63   
4_payment_confirmation_page            189.0    173.0    44.0    46.0    1.39   

                                                     
month                       2015-02 2015-03 2015-04  
page                                                 
2_search_page                100.00  100.00  100.00  
3_payment_page                17.62    7.11    6.57  
4_payment_confirmation_page    1.26    0.50    0.51